### Publish Model API

In [1]:
#set up API
import requests
import json
import os

os.environ['DOMINO_USER_API_KEY'] = "e96911141303c50867a8f331b4abdf0ca9f7719e7111b605d87fcec7d6b5ca29"
os.environ['DOMINO_API_HOST'] = "https://cdc-sandbox.domino-eval.com"

#set up API key and headers
api_key = os.environ['DOMINO_USER_API_KEY']
headers = {'X-Domino-Api-Key': api_key,  'Content-Type': 'application/json'}  

#api host
host = os.environ['DOMINO_API_HOST']

In [2]:

host

'https://cdc-sandbox.domino-eval.com'

In [3]:
import os


    
from domino import Domino

domino = Domino(
    "pok8/Heart-Attack-prediction",
    api_key="e96911141303c50867a8f331b4abdf0ca9f7719e7111b605d87fcec7d6b5ca29",
    host="https://cdc-sandbox.domino-eval.com"
)

os.environ['DOMINO_PROJECT_NAME']

'Heart-Attack-prediction'

In [4]:
#get user id
url_user = '{host}/v4/users/self'.format(host=host)
r_user = requests.get(url_user, headers=headers)
user_id = r_user.json()['id']

#get project id
project_name = os.environ['DOMINO_PROJECT_NAME']
url_project = '{host}/v4/projects?name={project_name}&ownerId={user_id}'.format(
    host=host, project_name=project_name, user_id=user_id)
r_project = requests.get(url_project, headers=headers)
projectId = r_project.json()[0]['id']
projectId

'650ddf9e5ab24c14cb26acaf'

In [5]:
#get default environment ID for the project
env_url = '{host}/v4/projects/{projectId}/settings'.format(host=host, projectId=projectId)
r_env = requests.get(env_url, headers=headers)
env_id = r_env.json()['defaultEnvironmentId']
env_id

'650255d38f2b224c6b1e2ab9'

In [6]:
#publish the model endpoint - Final one

## make sure to fill out below info ##
filename = 'heart-attack-predict.py'
function = 'predict'
model_name = 'lgb_heart_attack_prediction_api'
model_desc = 'Created using auto deploy'
commitId =  "ae5eb1bc5d5df130b9e70c1cce9d3930a9eb262d"
# Brand new version
publish_url = '{host}/v1/models'.format(host=host)
modelSource='Registry'

model_id= '6536d231f66ed97e65981017'
# Append new version
publish_url = '{host}/v1/models/{model_id}/versions'.format(host=host, model_id=model_id)

details = {
  "projectId": projectId,
  "environmentId": env_id,
  "name": model_name,
  "description": model_desc,
    
  'modelSource': 'lgb_heart_attack_prediction_api',
  'registeredModelName': 'lgb_heart_attack_model',
  'registeredModelVersion':'7'


}

r = requests.post(publish_url, headers=headers, json=details)
r.json()

{'id': '653826e0f66ed97e6598115a', 'number': 2}

## Export Model API to External Registry

In [21]:
#list model API in the current project
project_name = os.environ['DOMINO_PROJECT_NAME']
username = os.environ['DOMINO_PROJECT_OWNER']
modelapis_url = '{host}/v1/projects/{username}/{project_name}/models'.format(host=host, username=username,
                                                                         project_name=project_name)
modelapis_request = requests.get(modelapiss_url, headers=headers)
modelapis = modelapis_request.json()['data']
print(modelapis)
modelapis_id = modelapis[0]['id']
print(modelapis_id)

NameError: name 'modelapiss_url' is not defined

In [ ]:
#list versions of a chosen model API
versions_url = '{host}/v1/models/{model_id}/versions'.format(host=host, model_id=model_id)
versions_request = requests.get(versions_url, headers=headers)
versions = []

for x in range(len(versions_request.json()['data'])):
    versions.append(versions_request.json()['data'][x]['_id'])

versions

In [ ]:
#set your model version ID - defaults to the first one
modelVersionId = versions[0]
modelVersionId

In [ ]:
#export model
export_url = '{host}/v4/models/{modelId}/{modelVersionId}/exportImageToRegistry'.format(host=host,
                                                            modelId=model_id, modelVersionId=modelVersionId)

#set export parameters
#examples
#      "registryUrl": "quay.io",
#      "repository": "andrea_lowe/exported_model",
#      "password": "<your token/password>",
#      "tag": "test",
#      "username": "andrea_lowe"

repo_password = os.environ['<password that you stored as an environment variable>']
tag = '<tag>'

values = {"registryUrl": "<url>", "repository": "<username/reponame>", "password": repo_password,
          "tag": tag, "username": "<username>"}

r = requests.post(export_url, headers=headers, json=values)
r.json()

In [ ]:
exportId = r.json()['exportId']
exportId

In [ ]:
#export status
status_url = '{host}/v4/models/{exportId}/getExportImageStatus'.format(host=host,exportId=exportId)
status_request = requests.get(status_url, headers=headers)
status = status_request.json()
status

## API example for using Model Registry

In [14]:
#list all registered models visible to you

models_url = '{host}/api/registeredmodels/beta/names'.format(host=host)
models_request = requests.get(models_url, headers=headers)
models = models_request.json()
models

{'items': ['Andrea-Test',
  'custom_heart_attack_model_api',
  'demo-1',
  'demo-2',
  'demo-test-01',
  'direct_heart_attack_model',
  'heart-attack-LR',
  'heart-attack-LR-model',
  'heart-attack-model-v2',
  'heart-attack-prediction-1',
  'heart-attack-random-forest-v1',
  'heart_attack_sklearn_RF',
  'lgb_heart_attack_model',
  'original-sklearn_model_api',
  'sk_learn_model_RF_1',
  'sklearn_RF_model',
  'SVM-digits',
  'Test'],
 'metadata': {'requestId': 'aab16e64-b0e4-4ded-9daf-d682278d7a58',
  'notices': [],
  'offset': 0,
  'limit': 0,
  'totalCount': 1}}

In [ ]:


{
    "projectId": "5a73a66cd4632584f7b0ed8d",
    "environmentId": "5a73a6906ece7a7aed984654",
    "name": "Wine Quality Model",
    "description": "A model of wine quality",
    'modelSource': 'Registry',
    'registeredModelName': 'DEMO-mlflow',
    'registeredModelVersion':'4'
}



In [4]:
! curl -X GET "https://cdc-sandbox.domino-eval.com/api/environments/beta/environments" -H "accept: application/json"


{"requestId":"543bfcd4-9aac-4975-a6ac-838bcf874588","errors":["java.lang.IllegalArgumentException: No user found for principal AnonymousPrincipal"]}

In [20]:
import requests
import json
import os

url = 'https://cdc-sandbox.domino-eval.com/api/environments/beta/environments'
payload = json.dumps(
    {

    'modelSource': 'Registry',
    'registeredModelName': 'DEMO-mlflow',
    'registeredModelVersion': '4'
    }
)
response = requests.request("Get", url, headers=headers)
print(response.text)
for ele in json.loads(response.text)["environments"] : 
    print(ele['id'], ele['name'])

{"environments":[{"id":"650362f75ab24c14cb26aa07","name":"Demo","visibility":"Private","owner":{"id":"65035a6f0a7f750eb5f73308","username":"domino-andrea","ownerType":"Individual"},"supportedClusters":[],"latestRevision":{"id":"650362f75ab24c14cb26aa09","number":1,"status":"Succeeded","availableTools":[{"name":"jupyter","title":"Jupyter (Python, R, Julia)","startScripts":["/opt/domino/workspaces/jupyter/start"],"iconUrl":"/assets/images/workspace-logos/Jupyter.svg","proxyConfig":{"internalPath":"/{{ownerUsername}}/{{projectName}}/{{sessionPathComponent}}/{{runId}}/{{#if pathToOpen}}tree/{{pathToOpen}}{{/if}}","port":8888,"rewrite":false,"requireSubdomain":false},"supportedFileExtensions":[".ipynb"]},{"name":"jupyterlab","title":"JupyterLab","startScripts":["/opt/domino/workspaces/jupyterlab/start"],"iconUrl":"/assets/images/workspace-logos/jupyterlab.svg","proxyConfig":{"internalPath":"/{{ownerUsername}}/{{projectName}}/{{sessionPathComponent}}/{{runId}}/{{#if pathToOpen}}tree/{{pathTo